## Use map on an image without navigation dimension

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
import hyperspy.api as hs

In [2]:
from inline_holo import ModifiedImage as MI
from scipy.misc import ascent

In [3]:
imgdata = ascent() / 2**8
Ny, Nx = imgdata.shape
#imgdata = imgdata * np.exp(1j*np.random.rand(Nx, Ny))

dx = 0.5
dy = 0.5
dict_x = {'size':Nx, 'name':'X', 'units':'d', 'scale':dx, 'offset':0}
dict_y = {'size':Ny, 'name':'Y', 'units':'d', 'scale':dy, 'offset':0}
img = MI(imgdata, axes=[dict_y, dict_x])
img.metadata.General.title = 'Stairs'

#Nz = 10
#dict_z = {'size':Nz, 'name':'Z', 'units':'d', 'scale':1, 'offset':0}
#imgdata = np.repeat(imgdata[None,...], Nz, 0)
#img = MI(imgdata, axes=[dict_z, dict_y, dict_x])

# radial integration
R = [None, 0.25, 0.5, 5., 25, 10]
radii = []
imgri = []
for io in R:
    radii.append(img.get_digitized_radius(io))
    imgri.append(img.integrate_radial(io, show_progressbar=False))

# angular integration
A = [None, 0.1, np.pi/8., 100, 25, 10]
angii = []
imgai = []
for io in A:
    angii.append(img.get_digitized_angle(io))
    imgai.append(img.integrate_angular(io, show_progressbar=False))

In [4]:
hs.plot.plot_images(angii, per_row=len(A))
hs.plot.plot_spectra(imgai, legend=[str(io) for io in A], line_style='steps', legend_picking=False)

hs.plot.plot_images(radii, per_row=len(R))
hs.plot.plot_spectra(imgri, legend=[str(io) for io in R], line_style='steps', legend_picking=False)

## Use map on an image with navigation dimension

In [5]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
import hyperspy.api as hs

In [6]:
from inline_holo import ModifiedImage as MI
from scipy.misc import face

In [7]:
imgdata = np.moveaxis(face(), -1, 0) / 2**8
Nz, Ny, Nx = imgdata.shape
#imgdata = imgdata * np.exp(1j*np.random.rand(Nx, Ny))
dx = 0.5
dy = 0.5
dz = 1
dict_x = {'size':Nx, 'name':'X', 'units':'d', 'scale':dx, 'offset':0}
dict_y = {'size':Ny, 'name':'Y', 'units':'d', 'scale':dy, 'offset':0}
dict_z = {'size':Nz, 'name':'Z', 'units':'d', 'scale':1, 'offset':0}
img = MI(imgdata, axes=[dict_z, dict_y, dict_x])
img.metadata.General.title = 'Racoon faces'

# radial integration
R = 20
radius = img.get_digitized_radius(R)
imgri = img.integrate_radial(R, show_progressbar=False)

# angular integration
A = np.pi / 8.
angle = img.get_digitized_angle(A)
imgai = img.integrate_angular(A, show_progressbar=False)

In [8]:
hs.plot.plot_images([img, angle, radius], aspect='equal')
hs.plot.plot_spectra(imgri, line_style='steps', legend_picking=False)
hs.plot.plot_spectra(imgai, line_style='steps', legend_picking=False)

## Use with an FFT

In [9]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
import hyperspy.api as hs

In [10]:
from inline_holo import ModifiedImage as MI
from scipy.misc import face

In [11]:
imgdata = np.moveaxis(face(), -1, 0) / 2**8
Nz, Ny, Nx = imgdata.shape
#imgdata = imgdata * np.exp(1j*np.random.rand(Nx, Ny))
dx = 0.5
dy = 0.5
dz = 1
dict_x = {'size':Nx, 'name':'X', 'units':'d', 'scale':dx, 'offset':0}
dict_y = {'size':Ny, 'name':'Y', 'units':'d', 'scale':dy, 'offset':0}
dict_z = {'size':Nz, 'name':'Z', 'units':'d', 'scale':1, 'offset':0}
img = MI(imgdata, axes=[dict_z, dict_y, dict_x])
img.metadata.General.title = 'Racoon faces'
img = img.set_pad((50, 50), mode='constant', constant_values=img.mean((1,2)))

# radial integration
imgfreq = np.log(img.fft(shifted=True).amplitude)
imgfreq = MI(imgfreq)

# radial integration
R = 100
radius = imgfreq.get_digitized_radius(R, shifted=False)
imgri = imgfreq.integrate_radial(R, False, show_progressbar=False)

# angular integration
A = np.pi / 360.
angle = imgfreq.get_digitized_angle(A, shifted=False)
imgai = imgfreq.integrate_angular(A, False, show_progressbar=False)

In [12]:
hs.plot.plot_images([imgfreq, angle, radius], aspect='equal')
hs.plot.plot_spectra(imgri, line_style='steps', legend_picking=False)
hs.plot.plot_spectra(imgai, line_style='steps', legend_picking=False)

## Gaussian filering and Fourier shell correlation 

In [13]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
import hyperspy.api as hs

In [14]:
from inline_holo import ModifiedImage as MI
from inline_holo import ComplexModifiedImage as CMI
from scipy.misc import face

In [15]:
imgdata = np.moveaxis(face(), -1, 0) / 2**8
Nz, Ny, Nx = imgdata.shape
#imgdata = imgdata * np.exp(1j*np.random.rand(Nx, Ny))
dx = 0.5
dy = 0.5
dz = 1
dict_x = {'size':Nx, 'name':'X', 'units':'nm', 'scale':dx, 'offset':0}
dict_y = {'size':Ny, 'name':'Y', 'units':'nm', 'scale':dy, 'offset':0}
dict_z = {'size':Nz, 'name':'Z', 'units':'nm', 'scale':1, 'offset':0}
img = MI(imgdata, axes=[dict_z, dict_y, dict_x])
img.metadata.General.title = 'Racoon face'

In [16]:
img.plot()

Filter the image

In [17]:
# TODO: map for padding
img_pad = img.set_pad((100, 100), mode='constant', constant_values=img.mean((1,2)))
img_fft = img_pad.fft(True)

klist  = [0.1, 0.3, 0.5]
xx, yy = [axi.axis for axi in img_fft.axes_manager.signal_axes]
kabs   = np.abs(xx[None, :] + 1j* yy[:, None])
gauss  = img_fft.real.deepcopy()
for io, kcut in enumerate(klist):
    gauss.data[io, ...] = np.exp( - (kabs*2.*np.pi)**2. / kcut**2.) 
    
gauss = MI(gauss)

foo = lambda idata, gdata: idata * gdata 
img_fft_gauss = img_fft.map(foo, gdata = gauss, inplace=False, show_progressbar=False)

img_fft_gauss.ifft()
img_gauss_pad = img_fft_gauss.ifft()
img_gauss_pad = MI(img_gauss_pad)
img_gauss = img_gauss_pad.remove_pad()

In [18]:
CMI(img_fft).plot(representation='polar')

In [19]:
gauss.plot()

In [20]:
# Show the filter in action
ikwargs = {'shifted':False, 'show_progressbar':False}
Ifft_pure = MI(np.log10(img_fft.amplitude)).integrate_radial(**ikwargs)
Ifft_filt = MI(np.log10(img_fft_gauss.amplitude)).integrate_radial(**ikwargs)
Igaussian = (gauss.integrate_radial(**ikwargs))

f, ax = plt.subplots()
pkwargs = {'colors':['r', 'g', 'b']}
hs.plot.plot_spectra(Ifft_pure, line_style='-', fig=f, ax=ax, **pkwargs)
hs.plot.plot_spectra(Ifft_filt, line_style='--', fig=f, ax=ax, **pkwargs)
axt = ax.twinx()
hs.plot.plot_spectra(Igaussian, line_style=':', fig=f, ax=axt, **pkwargs)
ax.set_ylim(-5., None)
ax.set_xlim(None, 0.6)
ax.set_ylabel('Radially integrated power spectrum, dB')
ax.set_xlabel('Spatial frequency, 1/nm')
ax.legend(['Pure signal', '_nolegend_', '_nolegend_', 
           'Filtered signal', '_nolegend_', '_nolegend_', 
           'Gauss', '_nolegend_', '_nolegend_'])
ax.set_title('Racoon faces power spectrum, \n red to blue filter threshold decreases')

C:\Users\aeljarrat\AppData\Local\conda\conda\envs\hyperspy_dev\lib\site-packages\ipykernel\__main__.py:4: RuntimeWarning: divide by zero encountered in log10
C:\Users\aeljarrat\AppData\Local\conda\conda\envs\hyperspy_dev\lib\site-packages\matplotlib\legend.py:593: UserWarning: The handle <matplotlib.lines.Line2D object at 0x000001B72FE8F978> has a label of '_nolegend_' which cannot be automatically added to the legend.
  'legend.'.format(handle, label))
C:\Users\aeljarrat\AppData\Local\conda\conda\envs\hyperspy_dev\lib\site-packages\matplotlib\legend.py:593: UserWarning: The handle <matplotlib.lines.Line2D object at 0x000001B732AACF28> has a label of '_nolegend_' which cannot be automatically added to the legend.
  'legend.'.format(handle, label))
C:\Users\aeljarrat\AppData\Local\conda\conda\envs\hyperspy_dev\lib\site-packages\matplotlib\legend.py:593: UserWarning: The handle <matplotlib.lines.Line2D object at 0x000001B736C65D68> has a label of '_nolegend_' which cannot be automaticall

Text(0.5,1,'Racoon faces power spectrum, \n red to blue filter threshold decreases')

In [21]:
hs.plot.plot_images([img, img_gauss])

Calculate FSC

In [22]:
bin_size = 512

obs_ft = img_gauss_pad.fft(True)
exp_ft = img_pad.fft(True)

#obs_ft = img_gauss.fft(True)
#exp_ft = img.fft(True)

# Get correlations
axdict = obs_ft.axes_manager.as_dictionary()
axdict = [axdict[keys] for keys in axdict.keys()]
exp_obs_xc = MI(np.real(obs_ft * np.conj(exp_ft)), axes=axdict)
obs_ac     = MI(obs_ft.amplitude**2, axes=axdict)

axdict = exp_ft.axes_manager.as_dictionary()
axdict = [axdict[keys] for keys in axdict.keys()]
exp_ac     = MI(exp_ft.amplitude**2, axes=axdict)

# wrong fsc
#fsc_img = exp_obs_xc / np.sqrt(exp_ac*obs_ac)
#fsc1 = fsc_img.integrate_radial(bin_size=bin_size, shifted=False)

# good fsc
xc  = exp_obs_xc.integrate_radial(bin_size=bin_size, shifted=False, show_progressbar=False)
ac1 = exp_ac.integrate_radial(bin_size=bin_size, shifted=False, show_progressbar=False)
ac2 = obs_ac.integrate_radial(bin_size=bin_size, shifted=False, show_progressbar=False)

fsc = xc / np.sqrt(ac1*ac2)

In [24]:
hs.plot.plot_spectra(fsc.isig[:0.6], line_style='steps')